In [1]:
USE_FULL_1900_DIM_MODEL = False # if True use 1900 dimensional model, else use 64 dimensional one.

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Set seeds
tf.set_random_seed(42)
np.random.seed(42)

if USE_FULL_1900_DIM_MODEL:
    # Sync relevant weight files
    !aws s3 sync --no-sign-request --quiet s3://unirep-public/1900_weights/ 1900_weights/
    
    # Import the mLSTM babbler model
    from unirep import babbler1900 as babbler
    
    # Where model weights are stored.
    MODEL_WEIGHT_PATH = "./1900_weights"
    
else:
    # Sync relevant weight files
    !aws s3 sync --no-sign-request --quiet s3://unirep-public/64_weights/ 64_weights/
    
    # Import the mLSTM babbler model
    from unirep import babbler64 as babbler
    
    # Where model weights are stored.
    MODEL_WEIGHT_PATH = "./64_weights"

In [3]:
batch_size = 50
b = babbler(batch_size=batch_size, model_path=MODEL_WEIGHT_PATH)

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
# Before you can train your model, 
sequences = []
with open("emi_R4_neg_seqs_1NotR_1.txt", "r") as source:
    with open("formatted.txt", "w") as destination:
        for i,seq in enumerate(source):
            seq = seq.strip()
            sequences.append(seq)
            if b.is_valid_seq(seq) and len(seq) < 275: 
                formatted = ",".join(map(str,b.format_seq(seq)))
                destination.write(formatted)
                destination.write('\n')

In [5]:
## 
average_hidden_list = []
final_hidden_list = []
hs_list = []
final_cell_list = []


num2 = range(0, 50)
x = 0
y = 50
for i in num2:
    num1 = range(x, y)
    for j in num1:
        avg_hidden, final_hidden, final_cell, hs_out = (b.get_rep_hs(sequences[j]))
        average_hidden_list.append(avg_hidden)
        final_hidden_list.append(final_hidden)
        final_cell_list.append(final_cell)
        hs_list.append(hs_out)
    print('rep')
    x = x + 50
    y = y + 50
    


rep
rep
rep
rep
rep
rep
rep
rep
rep
rep


IndexError: list index out of range

In [6]:
average_hidden_pd = pd.DataFrame(np.row_stack(average_hidden_list))
final_hidden_pd = pd.DataFrame(np.row_stack(final_hidden_list))
hidden_state = pd.DataFrame(np.row_stack(hs_list))
print(average_hidden_pd)

           0         1         2         3         4         5         6   \
0    0.020637  0.158141 -0.095418 -0.960367 -0.011774 -0.211894  0.061429   
1    0.036258  0.160610 -0.095683 -0.955451 -0.008399 -0.241450  0.063239   
2    0.048274  0.180363 -0.096080 -0.957145 -0.012548 -0.229441  0.061430   
3    0.029994  0.164992 -0.095810 -0.959701 -0.011470 -0.252243  0.069592   
4    0.036875  0.179269 -0.097630 -0.957563 -0.011411 -0.241152  0.070157   
5    0.039135  0.172686 -0.094565 -0.955958 -0.006718 -0.250925  0.062368   
6    0.018737  0.143936 -0.096921 -0.959309 -0.010651 -0.195520  0.054121   
7    0.037273  0.169321 -0.097265 -0.955835 -0.010341 -0.238991  0.063187   
8    0.040586  0.181305 -0.094121 -0.956921 -0.008719 -0.249279  0.066645   
9    0.037736  0.169447 -0.092086 -0.957993 -0.007213 -0.253726  0.066040   
10   0.029900  0.174069 -0.106624 -0.955552 -0.016978 -0.249228  0.069885   
11   0.025923  0.160529 -0.092726 -0.958151 -0.004918 -0.218862  0.055149   

In [7]:
average_hidden_pd.to_csv("emi_R4_neg_reps_1NotR_1.csv")

In [8]:
import pickle
save_loc = "C:\\Users\\makow\\Documents\\GitHub\\UniRep\\Datasets"
data_name = 'ins_seq_1'
file_append = '.pickle'


fn = save_loc + data_name + 'avg_hidden' + file_append
with open(fn, 'wb') as f:
    pickle.dump(average_hidden_list, f)

fn = save_loc + data_name + 'final_hidden' + file_append
with open(fn, 'wb') as f:
    pickle.dump(final_hidden_list, f)

fn = save_loc + data_name + 'final_cell' + file_append
with open(fn, 'wb') as f:
    pickle.dump(final_cell_list, f)
   
fn = save_loc + data_name + 'hidden_state' + file_append
with open(fn, 'wb') as f:
    pickle.dump(hs_list, f)
   
fn = save_loc + data_name + 'all_output_hs' + file_append
with open(fn, 'wb') as f:
    pickle.dump([average_hidden_list, final_hidden_list, final_cell_list, hs_list], f)